In [1]:
#Neural Network basics: perceptron
# We set weights (per input) and a bias by using random numbers as guess 
# We start with 2 input values (Height and Weight)
# We mulitiply our input values with the weights and add up the values 
# We add the bias to these values
# we now need a classification - male or female = we need and activator function 
# we are using sigmoid (easy to learn, not great to use has problems)
# sigmoid's job is to convert what ever the summation ( sum weights* iput) into a 0 or 1 
# video on Teams explaining why this works  - ie the limits of sigmoid 
# create sigmoid and sigmoid derivitive function ( derivite = rate of change )
# get first prediction - apply cost/loss/error function ( how wrong is our first guess - relates to gradient descent)
# decide on learning rate - (to do with gradient descent - it will update the weights and biases by this rate)

# Building a perceptron - the basics of Neural Networks
## Overview of steps we are taking:


In [7]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.mllib.evaluation import BinaryClassificationMetrics, RegressionMetrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, RFormula
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import matplotlib.pyplot as plt
import numpy as np
import datetime
import atexit
from pyspark.sql import functions

In [4]:
from pyspark import SparkContext, SparkConf
conf_spark = SparkConf().set("spark.driver.host", "127.0.0.1")
sc = SparkContext(conf=conf_spark)

In [5]:
spark = SparkSession.builder.getOrCreate()

In [6]:
df = spark.createDataFrame([('Male', 67, 150), # insert column values
('Female', 65, 135),
('Female', 68, 130),
('Male', 70, 160),
('Female', 70, 130),
('Male', 69, 174),
('Female', 65, 126),
('Male', 74, 188),
('Female', 60, 110),
('Female', 63, 125),
('Male', 70, 173),
('Male', 70, 145),
('Male', 68, 175),
('Female', 65, 123),
('Male', 71, 145),
('Male', 74, 160),
('Female', 64, 135),
('Male', 71, 175),
('Male', 67, 145),
('Female', 67, 130),
('Male', 70, 162),
('Female', 64, 107),
('Male', 70, 175),
('Female', 64, 130),
('Male', 66, 163),
('Female', 63, 137),
('Male', 65, 165),
('Female', 65, 130),
('Female', 64, 109)],
['gender', 'height','weight']) # insert header values

In [8]:
df= df.withColumn('gender',functions.when(df['gender']=='Female',0).otherwise(1))

In [9]:
df = df.select('height','weight','gender')

In [10]:
df.show()

+------+------+------+
|height|weight|gender|
+------+------+------+
|    67|   150|     1|
|    65|   135|     0|
|    68|   130|     0|
|    70|   160|     1|
|    70|   130|     0|
|    69|   174|     1|
|    65|   126|     0|
|    74|   188|     1|
|    60|   110|     0|
|    63|   125|     0|
|    70|   173|     1|
|    70|   145|     1|
|    68|   175|     1|
|    65|   123|     0|
|    71|   145|     1|
|    74|   160|     1|
|    64|   135|     0|
|    71|   175|     1|
|    67|   145|     1|
|    67|   130|     0|
+------+------+------+
only showing top 20 rows



In [11]:
df.select("height","weight","gender").collect()

[Row(height=67, weight=150, gender=1),
 Row(height=65, weight=135, gender=0),
 Row(height=68, weight=130, gender=0),
 Row(height=70, weight=160, gender=1),
 Row(height=70, weight=130, gender=0),
 Row(height=69, weight=174, gender=1),
 Row(height=65, weight=126, gender=0),
 Row(height=74, weight=188, gender=1),
 Row(height=60, weight=110, gender=0),
 Row(height=63, weight=125, gender=0),
 Row(height=70, weight=173, gender=1),
 Row(height=70, weight=145, gender=1),
 Row(height=68, weight=175, gender=1),
 Row(height=65, weight=123, gender=0),
 Row(height=71, weight=145, gender=1),
 Row(height=74, weight=160, gender=1),
 Row(height=64, weight=135, gender=0),
 Row(height=71, weight=175, gender=1),
 Row(height=67, weight=145, gender=1),
 Row(height=67, weight=130, gender=0),
 Row(height=70, weight=162, gender=1),
 Row(height=64, weight=107, gender=0),
 Row(height=70, weight=175, gender=1),
 Row(height=64, weight=130, gender=0),
 Row(height=66, weight=163, gender=1),
 Row(height=63, weight=13

In [13]:
data_array = np.array(df.select(["height","weight","gender"]).collect())

In [ ]:
np.random.seed(12345)

In [14]:
w1=np.random.randn()
w2=np.random.randn()
b= np.random.randn()

In [15]:
print(w1,w2,b)

1.3969417500646113 -0.49969513509521685 0.6564454369426809


In [17]:
X= data_array[:,:2]
y = data_array[:,2]

In [18]:
x_mean = X.mean(axis=0)
x_std = X.std(axis=0)

In [27]:
def normalize(X):
    x_mean= X.mean(axis=0)
    x_std=X.std(axis=0)
    X=(X-X.mean(axis=0))/ X.std(axis=0)
    return X

In [29]:
X=normalize(X)
print(X)

[[-0.06163661  0.21460055]
 [-0.65745714 -0.4618577 ]
 [ 0.23627366 -0.68734378]
 [ 0.8320942   0.66557271]
 [ 0.8320942  -0.68734378]
 [ 0.53418393  1.29693375]
 [-0.65745714 -0.86773265]
 [ 2.02373527  1.92829478]
 [-2.14700848 -1.58928812]
 [-1.25327768 -0.91282987]
 [ 0.8320942   1.25183653]
 [ 0.8320942  -0.01088554]
 [ 0.23627366  1.34203096]
 [-0.65745714 -1.0030243 ]
 [ 1.13000446 -0.01088554]
 [ 2.02373527  0.66557271]
 [-0.95536741 -0.4618577 ]
 [ 1.13000446  1.34203096]
 [-0.06163661 -0.01088554]
 [-0.06163661 -0.68734378]
 [ 0.8320942   0.75576715]
 [-0.95536741 -1.72457977]
 [ 0.8320942   1.34203096]
 [-0.95536741 -0.68734378]
 [-0.35954687  0.80086436]
 [-1.25327768 -0.37166327]
 [-0.65745714  0.8910588 ]
 [-0.65745714 -0.68734378]
 [-0.95536741 -1.63438533]]


In [32]:
print('standard deviation')
print(round(X[:,0].std(axis=0),0))
print('mean')
print(round(X[:,0].mean(axis=0),0))

standard deviation
1.0
mean
0.0


In [33]:
def sigmoid(input):
    return 1/(1+np.exp(-input))

In [34]:
def sigmoid_derivative(x):
    return sigmoid(x) * (1-sigmoid(x))

In [ ]:
def predict_gender(score):
    gender_summation = score[0]*w1+ score[1]*w2 +b
    gender_score=sigmoid(gender_summation)
    if gender_score <= 0.5:
        gender ='Female'
    else:
        gender= 'Male'

    return gender, gender_score
